<h1 align="center">A Brief Introduction to BERT</h1>

Data Scientist.: Dr.Eddy Giusepe Chirinos Isidro

# Introdução

À medida que aprendemos [o que é um Transformer](https://machinelearningmastery.com/the-transformer-model/) e como podemos [treinar o modelo Transformer](https://machinelearningmastery.com/training-the-transformer-model/), percebemos que é uma ótima ferramenta para fazer um computador entender a **linguagem humana**. No entanto, o `Transformer` foi originalmente projetado como um modelo para traduzir um idioma para outro. Se o redirecionarmos para uma tarefa diferente, provavelmente precisaremos treinar novamente todo o modelo do zero. Dado que o tempo necessário para treinar um modelo de `Transformer` é enorme, gostaríamos de ter uma solução que nos permitisse reutilizar prontamente o Transformer treinado para muitas tarefas diferentes. O `BERT` é um desses modelos. É uma extensão da parte do `encoder` (codificador) de um transformador.

Neste script, você aprenderá o que é o `BERT` e descobrirá o que ele pode fazer. Os pontos a estudar são:

* O que é um `Bidirectional Encoder Representations de Transformer` (<font color="yellow">BERT</font>)

* Como um modelo `BERT` pode ser reutilizado para diferentes propósitos

* Como você pode usar um modelo `BERT pré-treinado`

# Do modelo de Transformer ao BERT

No modelo do `Transformer`, o Encoder (codificador) e o Decoder (decodificador) são conectados para fazer um `modelo seq2seq` para que você faça uma tradução, como do inglês para o alemão, etc. Lembre-se de que a equação da atenção diz:

$$
attention(Q, K, V) = softmax(\frac{Q K^T}{\sqrt{d_k}})V,
$$

onde $Q$, $K$ e $V$ acima é um vetor EMBEDDING transformado por uma matriz de peso no modelo do Transformer. Treinar um modelo de transformer significa encontrar essas **matrizes de peso**. Uma vez que as matrizes de peso são aprendidas, o transformer se torna um **modelo de linguagem**, o que significa que representa uma maneira de entender a linguagem que você usou para treiná-lo.


![](https://wikidocs.net/images/page/162096/3_encoder_decoder_class.png)


Um transformer possui partes codificadoras (`Encoder`) e decodificadoras (`Decoder`). Como o nome indica, o <font color="yellow">codificador</font> transforma frases e parágrafos em um formato interno (`uma matriz numérica`) que entende o contexto, enquanto o <font color="yellow">decodificador</font> faz o inverso. A combinação do codificador e do decodificador permite que um transformer execute tarefas `seq2seq`, como `tradução`. Se você retirar a parte do codificador do Transformer, ela pode lhe dizer algo sobre o contexto, o que pode fazer algo interessante.

A `representação do codificador bidirecional do Transformer` (BERT) aproveita o modelo de atenção (Attention Model) para obter uma compreensão mais profunda do contexto da linguagem. <font color="orange">`BERT` é uma pilha de muitos blocos codificadores (Encoder)</font>. O texto de entrada é separado em tokens como no modelo do Transformer, e cada token será transformado em um vetor na saída do BERT.

# O que o BERT pode fazer?

Um modelo `BERT` é treinado usando o <font color="orange">modelo de linguagem mascarada (MLM)</font> e <font color="yellow">a previsão da próxima sentença (NSP)</font> simultaneamente.

![](https://machinelearningmastery.com/wp-content/uploads/2022/10/BERT.png)

Cada amostra de treinamento para `BERT` é um par de frases (sentences) de um documento. As duas frases podem ser consecutivas no documento ou não. Haverá um token `[CLS]` anexado à primeira frase (para representar a **classe**) e um token `[SEP]` anexado a cada frase (como um **separador**). Em seguida, as duas sentenças serão concatenadas como uma sequência de tokens para se tornar uma amostra de treinamento. Uma pequena porcentagem dos tokens na amostra de treinamento é `mascarada` (*masked*) com um token especial `[MASK]` ou substituída por um token aleatório.

Antes de ser alimentado no modelo `BERT`, os tokens na amostra de treinamento serão transformados em `vetores EMBEDDINGS`, com as codificações (encodings) posicionais adicionadas, e particulares ao `BERT`, com `segmentos de Embeddings` adicionados, bem como para marcar se o token é da primeira ou da segunda frase.


Cada token de entrada para o modelo `BERT` produzirá um vetor de saída. Em um modelo BERT bem treinado, esperamos:

* <font color="orange">saída correspondente ao token mascarado pode revelar qual era o token original</font>

* <font color="orange">a saída correspondente ao token `[CLS]` no início pode revelar se as duas sentenças são consecutivas no documento</font>


Então, os pesos treinados no modelo `BERT` podem entender bem o contexto da linguagem.


Depois de ter esse modelo de BERT, você pode usá-lo para muitas `tarefas de downstream`. <font color="yellow">Por exemplo</font>, adicionando uma camada de `classificação` apropriada em cima de um codificador (`Encoder`) e alimentando apenas uma frase ao modelo em vez de um par, você pode usar o token de classe `[CLS]` como entrada para `classificação de sentimento`. Funciona porque a saída do token de classe é treinada para agregar a atenção para toda a entrada.

Outro <font color="yellow">exemplo</font> é tomar uma pergunta como a primeira frase e o texto (por exemplo, um parágrafo) como a segunda frase, então o token de saída da segunda frase pode marcar a posição onde a resposta à pergunta estava. Funciona porque a saída de cada token revela algumas informações sobre esse token no contexto de toda a entrada.

# Usando o modelo BERT pré-treinado para sumarização

Um modelo de transformer leva muito tempo para treinar do zero. O modelo `BERT` levaria ainda mais tempo. Mas o objetivo do `BERT` é criar um modelo que possa ser reutilizado para muitas tarefas diferentes.

Existem modelos `BERT pré-treinados` que você pode usar prontamente. A seguir, você verá alguns casos de uso. O texto usado no exemplo a seguir é de:

* [The Bank of England's imperfect Intervention](https://www.project-syndicate.org/commentary/bank-of-england-gilt-purchases-necessary-but-mistakes-made-by-willem-h-buiter-and-anne-c-sibert-2022-10?barrier=accesspaylog)


Teoricamente, um modelo BERT é um codificador (`Encoder`) que mapeia cada token de entrada para um vetor de saída, que pode ser estendido para uma sequência de tokens de comprimento infinito. **Na prática**, existem limitações impostas na implementação de outros componentes que limitam o tamanho da entrada (**input size**). Principalmente, algumas centenas de tokens devem funcionar, pois nem toda implementação pode receber milhares de tokens de uma só vez. Você pode salvar o artigo inteiro em *article.txt* (uma cópia está disponível [aqui](https://machinelearningmastery.com/wp-content/uploads/2022/10/article.txt)). Caso seu modelo precise de um texto menor, você pode usar apenas alguns parágrafos dele.

<font color="yellow">Primeiro</font>, vamos explorar a tarefa de `sumarização`. Usando o `BERT`, a ideia é extrair algumas frases do texto original que representem todo o texto. Você pode ver que essa tarefa é semelhante à previsão da próxima frase, na qual, se receber uma frase e o texto, você deseja classificar se eles estão relacionados.



Para fazer isso, você precisa usar o módulo Python `bert-extractive-summarizer`.

In [ ]:
%pip install bert-extractive-summarizer

É um wrapper para alguns modelos `Hugging Face` para fornecer o pipeline de tarefas de `sumarização`. Hugging Face é uma plataforma que permite publicar modelos de aprendizado de máquina, principalmente em tarefas de `NLP`.

Depois de instalar o `bert-extractive-summarizer`, produzir um resumo é apenas algumas linhas de código:

In [4]:
from summarizer import Summarizer

text = open("article.txt").read()
text

'Amid the political turmoil of outgoing British Prime Minister Liz Truss’s\nshort-lived government, the Bank of England has found itself in the\nfiscal-financial crossfire. Whatever government comes next, it is vital\nthat the BOE learns the right lessons.\n\nOn September 23, Truss’s government announced a large, unfunded\nfiscal-stimulus package that undermined the BOE’s price-stability mandate\nand caused government-debt and foreign-exchange markets to malfunction.\nTo prevent systemically important markets from seizing up, the BOE\npostponed its planned offloading of government bonds (gilts) and instead\npurchased more. But these unsterilized (monetized) asset purchases amount\nto an expansionary monetary policy, which will further frustrate the BOE’s\nefforts to bring down inflation – unless and until the purchases are\nreversed.\n\nUnfortunately for the BOE, it had no choice. Financial stability is a\nprecondition for sustainable price stability, so it is a central bank’s\noverrid

In [6]:
model = Summarizer('distilbert-base-uncased')

result = model(text, num_sentences=3)

print(result)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Amid the political turmoil of outgoing British Prime Minister Liz Truss’s
short-lived government, the Bank of England has found itself in the
fiscal-financial crossfire. Whatever government comes next, it is vital
that the BOE learns the right lessons. According to a statement by the BOE’s Deputy Governor for
Financial Stability, Jon Cunliffe, the MPC was merely “informed of the
issues in the gilt market and briefed in advance of the operation,
including its financial-stability rationale and the temporary and targeted
nature of the purchases.”


Esse é o código completo! 

Nos bastidores, o `spaCy` foi usado em alguns pré-processamentos e o `Hugging Face` foi usado para lançar o modelo. O modelo utilizado foi denominado `distilbert-base-uncased`. O `DistilBERT` é um modelo `BERT` simplificado que pode rodar mais rápido e usar menos memória. O modelo é `“sem maiúsculas”`, o que significa que as maiúsculas ou minúsculas no texto de entrada são consideradas as mesmas, uma vez que são transformadas em vetores EMBEDDINGS.

A saída do modelo do `summarizer` é uma string. Conforme você especificou `num_sentences=3` ao chamar o modelo, o resumo são três frases selecionadas do texto. Essa abordagem é chamada de <font color="yellow">resumo extrativo</font>. A alternativa é um <font color="yellow">resumo abstrativo</font>, no qual o resumo é gerado ao invés de extraído do texto. Isso precisaria de um modelo diferente do `BERT`.

# Usando o modelo BERT pré-treinado para responder a perguntas

O outro exemplo de uso do `BERT` é combinar perguntas com respostas. Você entregará a pergunta e o texto ao modelo e procurará a saída do início e do fim da resposta do texto.

Um exemplo rápido seria apenas algumas linhas de código como segue, reutilizando o mesmo texto de exemplo do exemplo anterior:

In [8]:
from transformers import pipeline

text = open("article.txt").read()

question = "What is BOE doing?"
 

answering = pipeline("question-answering", model='distilbert-base-uncased-distilled-squad')

result = answering(question=question, context=text)

print(result)

{'score': 0.42956647276878357, 'start': 1261, 'end': 1344, 'answer': 'to maintain or restore market liquidity in systemically important\nfinancial markets'}


Aqui, `Hugging Face` é usado diretamente. Se você instalou o módulo usado no exemplo anterior, o módulo `Hugging Face Python` é uma dependência que você já instalou. Caso contrário, pode ser necessário instalá-lo com pip:

```
%pip install transformers
```

E para realmente usar um `modelo Hugging Face`, você também deve ter o `PyTorch` e o `TensorFlow` instalados:

```
%pip install torch tensorflow
```

A saída do código acima é um `dicionário Python`, como segue:

![](./saida_question_answer.png)

É aqui que você pode encontrar a resposta (que é uma frase do texto de entrada), bem como a posição inicial e final na ordem do token de onde veio essa resposta. A pontuação pode ser considerada como a pontuação de confiança do modelo de que a resposta pode se encaixar na pergunta.

Nos bastidores, o que o modelo fez foi gerar uma pontuação de probabilidade para o melhor início do texto que responde à pergunta, bem como o texto para o melhor final. Em seguida, a resposta é extraída encontrando a localização das maiores probabilidades.